In [ ]:
%pip install transformers sentence-transformers torch

In [2]:
from sentence_transformers import SentenceTransformer, models
from transformers import BertTokenizer,AutoTokenizer
import torch

import numpy as np
import pandas as pd

/home/mijanur/Desktop/AI/DL NLP/natural_language_processing/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [4]:
t1 = 'I like to eat apples.'
t2 = 'I like to play.'

In [5]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [6]:
# bert-base-uncased
class Bert(torch.nn.Module):
    def __init__(self):
        super(Bert, self).__init__()
        self.bert = models.Transformer('sentence-transformers/all-MiniLM-L6-v2', max_seq_length=128)
        self.pooling_layer = models.Pooling(self.bert.get_word_embedding_dimension())
        self.sts_bert = SentenceTransformer(modules=[self.bert, self.pooling_layer])

    def forward(self, input_data):
        output = self.sts_bert(input_data)['sentence_embedding']
        return output

In [7]:
# PATH = 'bert-sts.pt'
PATH ="bert-sts-org.pt"

In [ ]:
from transformers import BertConfig, BertModel

# Replace 'bert-base-uncased' with the correct pre-trained model identifier
config = BertConfig.from_pretrained('bert-base-uncased', hidden_size=768)

# Initialize your BERT model with the specified configuration
model = BertModel(config)

# Load the pre-trained weights from checkpoint
model.load_state_dict(torch.load('bert-sts-org.pt'))

In [ ]:
model = Bert()
model.load_state_dict(torch.load(PATH), strict=False)
model.eval()

In [10]:
model.eval()

def predict_similarity(sentence_pair):
  test_input = tokenizer(sentence_pair, padding='max_length', max_length = 128, truncation=True, return_tensors="pt").to(device)
  test_input['input_ids'] = test_input['input_ids']
  test_input['attention_mask'] = test_input['attention_mask']
  del test_input['token_type_ids']
  output = model(test_input)
  similarity = torch.nn.functional.cosine_similarity(output[0], output[1], dim=0).item()

  return similarity

In [13]:
predict_similarity([t1,t2])

0.4813702702522278

In [14]:
text = "Globalization is a multifaceted process involving the increasing interconnectedness and interdependence of people, economies, cultures, and societies across the world. It is facilitated by advancements in communication, transportation, and technology, allowing for the rapid exchange of goods, services, information, and ideas across borders.\n\nGlobalization has various dimensions:\n\n1. Economic Globalization: Involves the integration of national economies into the global economy through trade, investment, and the flow of capital.\n\n2. Cultural Globalization: Refers to the spread of cultural products, values, and practices around the world, leading to cultural homogenization and hybridization.\n\n3. Technological Globalization: Involves the diffusion and adoption of technologies worldwide, shaping communication and information-sharing.\n\n4. Political Globalization: Encompasses the growing influence of international organizations, treaties, and agreements in shaping global governance.\n\nGlobalization has both positive and negative consequences. On one hand, it has facilitated economic growth, cultural exchange, and technological advancements. On the other hand, it has also raised concerns about inequality, exploitation, cultural erosion, and environmental impacts.\n\nUnderstanding globalization is crucial in analyzing global issues, international cooperation, and the challenges of an interconnected world."

In [15]:
len(text.split())

176